In [18]:
import pandas as pd
import numpy as np
import os
import music21 as tune
import json
import keras

In [19]:
MAPPING_PATH='/content/mapping/mappings'
SINGLE_PATH='/content/allsongs2/all (1).txt'

In [20]:
# To load the dataset and perform Exploratory Dataset Analysis
def import_dataset(data):
  songs=[]
  for path,dir,files in os.walk(data):
    for file in files:
      if file[-3:]=='krn':
          song=tune.converter.parse(os.path.join(path,file))
          songs.append(song)
  return songs

In [21]:
!unzip /content/AudioSample.zip

Streaming output truncated to the last 5000 lines.
  inflating: essen/europa/deutschl/zuccal/deut5063.krn  
  inflating: essen/europa/deutschl/zuccal/deut4683.krn  
  inflating: essen/europa/deutschl/zuccal/deut4598.krn  
  inflating: essen/europa/deutschl/zuccal/deut4638.krn  
  inflating: essen/europa/deutschl/zuccal/deut4929.krn  
  inflating: essen/europa/deutschl/zuccal/deut4608.krn  
  inflating: essen/europa/deutschl/zuccal/deut5067.krn  
  inflating: essen/europa/deutschl/zuccal/deut4567.krn  
  inflating: essen/europa/deutschl/zuccal/deut4615.krn  
  inflating: essen/europa/deutschl/zuccal/deut4959.krn  
  inflating: essen/europa/deutschl/zuccal/deut5106.krn  
  inflating: essen/europa/deutschl/zuccal/deut4597.krn  
  inflating: essen/europa/deutschl/zuccal/deut4738.krn  
  inflating: essen/europa/deutschl/zuccal/deut4706.krn  
  inflating: essen/europa/deutschl/zuccal/deut4847.krn  
  inflating: essen/europa/deutschl/zuccal/deut4778.krn  
  inflating: essen/europa/deutschl/zu

In [22]:

dataset="/content/essen/europa/deutschl/test"
songs=import_dataset(dataset)
print("Succesfully imported songs :",len(songs))


Succesfully imported songs : 12


In [23]:
def eda(song):

  timestamp=[]
  for i in range(20):
    timestamp.append(i*0.25)
  for note in song.flat.notesAndRests:
    if note.duration.quarterLength not in timestamp:
      return False
  return True


In [24]:
def reutrn_songs(songs):
    count=0
    for song_iter in songs:
      if not(eda(song_iter)):
        count+=1
        continue
    song_iter=preprocess(song_iter)


In [25]:
def preprocess(song):


  # Checking if there is already existing key in the note
  music=song.getElementsByClass(tune.stream.Part)
  measure_part=music[0].getElementsByClass(tune.stream.Measure)
  key=measure_part[0][4]
  # Setting a value of the key if unavailable
  if not isinstance(key,tune.key.Key):
    key=song.analyze('key')
  # Setting preprocessing of notes from different pitch
  if key.mode=='major':
    l=tune.interval.Interval(key.tonic,tune.pitch.Pitch("C"))
  elif key.mode=='minor':
    l=tune.interval.Interval(key.tonic,tune.pitch.Pitch("B"))
  preprocessed_result=song.transpose(l)
  return preprocessed_result


In [26]:
# Reducing the dataset to only two notes to reduce computation
preprocessed_songs=[]
for song in songs:
  song=preprocess(song)
  preprocessed_songs.append(song)
print(len(preprocessed_songs))


12


In [27]:
type(song)

music21.stream.base.Score

In [28]:
def encoder(song,step_interval=0.25):
  encoded_song=[]
  for new in song.flat.notesAndRests:

    #convert the songs in readable format
    if isinstance(new,tune.note.Note):
      symbol=new.pitch.midi
    elif isinstance(new,tune.note.Rest):
      symbol='r'
    newnote=int(new.duration.quarterLength/step_interval)
    for j in range(newnote):
      if j == 0 :
        encoded_song.append(symbol)
      else:
        encoded_song.append('_')
  encoded_song=" ".join(map(str,encoded_song))
  return encoded_song




In [29]:
def edadata(dataset_path):
  songs=import_dataset(dataset_path)
  print("Succesfully loaded songs :",len(songs))
  songs_new=""
  for i, song in enumerate(songs):
    if not eda(song):
      continue
    #preprocessing songs
    song=preprocess(song)


    #time-series
    encoded_song=encoder(song)

    # text file
    sequence_len=64
    new_delimiter='/ '*sequence_len
    songs_new=songs_new+encoded_song+new_delimiter

  songs_new=songs_new[:-1]
  file_path="all"+".txt"

  path_songs=os.path.join('/content/allsongs2',file_path)
  with open(path_songs,"w") as fp:
    fp.write(songs_new)
  return songs_new


In [30]:
import json
def create_mappings(songs,mapping_path):
  mappings={}
  songs=songs.split()
  vocabulary=list(set(songs))
  for i,symbol in enumerate(vocabulary):
    mappings[symbol]=i
  with open(mapping_path,"w") as fp:
    json.dump(mappings,fp,indent=4)

In [31]:
dataset="/content/essen/europa/deutschl/test"
song_to_json=edadata(dataset)
file_path="mapping.txt"

create_mappings(song_to_json,'/content/mapping/mappings')

Succesfully loaded songs : 12


/usr/local/lib/python3.10/dist-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [32]:

def convert_songs_to_int(songs):
    int_songs = []

    # load mappings
    with open(MAPPING_PATH, "r") as fp:
        mappings = json.load(fp)

    # transform songs string to list
    songs = songs.split()

    # map songs to int
    for symbol in songs:
        int_songs.append(mappings[symbol])

    return int_songs

In [33]:
def load(file_path):
    with open(file_path, "r") as fp:
        song = fp.read()
    return song


In [34]:

def training_sequences(sequence_length):

    # load songs and map them to int
    songs2 = load(SINGLE_PATH)
    int_songs = convert_songs_to_int(songs2)

    inputs = []
    targets = []

    # generate the training sequences
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])

    # one-hot encode the sequences
    vocabulary_size = len(set(int_songs))
    # inputs size: (# of sequences, sequence length, vocabulary size)
    inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
    targets = np.array(targets)

    print(f"There are {len(inputs)} sequences.")

    return inputs, targets


In [35]:
inputs,targets=training_sequences(64)
print(targets)
print(inputs)

There are 2500 sequences.
[ 9  8  2 ... 15 15 15]
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ..

In [36]:
import keras

OUTPUT_UNITS = 23
NUM_UNITS = [256]
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
EPOCHS = 46
BATCH_SIZE = 64
MODEL_PATH = "/content/model (3).h5"
SEQUENCE_LENGTH=64

def build_model(output_units, num_units, loss, learning_rate):

    # create the model architecture
    input = keras.layers.Input(shape=(None, output_units))
    x = keras.layers.LSTM(num_units[0])(input)
    x = keras.layers.Dropout(0.2)(x)

    output = keras.layers.Dense(output_units, activation="softmax")(x)

    model = keras.Model(input, output)

    # compile model
    model.compile(loss=loss,
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  metrics=["accuracy"])

    model.summary()

    return model


def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):

    # generate the training sequences
    inputs, targets = training_sequences(SEQUENCE_LENGTH)

    # build the network
    model = build_model(output_units, num_units, loss, learning_rate)

    # train the model
    model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)

    # save the model
    model.save(MODEL_PATH)

train()

There are 2500 sequences.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 23)]        0         
                                                                 
 lstm (LSTM)                 (None, 256)               286720    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 23)                5911      
                                                                 
Total params: 292631 (1.12 MB)
Trainable params: 292631 (1.12 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/46
40/40 [==============================] - 14s 290ms/step - loss: 1.6728 - accuracy: 0.6588
Epoch 2/46
40/40 [============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [38]:
import json
import numpy as np

class MelodyGenerator:

    def __init__(self, model_path="/content/model (3).h5"):
        """Constructor that initialises TensorFlow model"""

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._startingvals = ["/"] * SEQUENCE_LENGTH


    def construct_melody_to_midi(self, seed, num_steps, max_sequence_length, temperature):

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._startingvals + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))

            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]

            output_int = self.sampletemperaturemaps(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def sampletemperaturemaps(self, probval, temperature):

        predictions = np.log(probval) / temperature
        probval = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probval)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probval)

        return index
    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="generated.mid"):


        stream = tune.stream.Stream()

        startingval = None
        counts = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if startingval is not None:

                    quarter_length_duration = step_duration * counts # 0.25 * 4 = 1

                    # handle rest
                    if startingval == "r":
                        tune_event = tune.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        tune_event = tune.note.Note(int(startingval), quarterLength=quarter_length_duration)

                    stream.append(tune_event)

                    # reset the step counter
                    counts = 1

                startingval = symbol

            else:
                counts += 1

        # write the tune stream to a midi file
        stream.write(format, file_name)





if __name__ == "__main__":
    mg = MelodyGenerator()
    seed = "55 _ _ _ 60 _ _ _ 55 _ _ _ 55 _"
    melody = mg.construct_melody_to_midi(seed, 500, SEQUENCE_LENGTH, 0.7)
    print(melody)

1/1 [==============================] - 0s 32ms/step
['55', '_', '_', '_', '60', '_', '_', '_', '55', '_', '_', '_', '55', '_', '_', '_', '62', '_', '62', '_', '62', '_', '_', '_', '_', '_', '60', '_', '_', '_', '55', '_', '_', '_', '65', '_', '_', '_', '65', '_', '65', '_', '65', '_', '64', '_', '64', '_', '67', '_', '64', '_', '_', '_', '62', '_', '_', '_', 'r', '_', '_', '_', '55', '_', '_', '_', '67', '_', '_', '_', '67', '_', '67', '_', '_', '_', '67', '_', '65', '_', '65', '_', '64', '_', '_', '_', '_', '_', '_', '_', '67', '_', '65', '_', '64', '_', '62', '_', '60', '_', '_', '_', '_', '_', '60', '_', '64', '_', '62', '_', '62', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '65', '_', '_', '_', '64', '_', '_', '_', '64', '_', '64', '_', '_', '_', '65', '_', '64', '_', '64', '_', '60', '_', '_', '_', '60', '_', '_', '_', 'r', '_', '_', '_/']


In [ ]:
mg = MelodyGenerator()
seed = "60 _ 64 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
seed2 = "55 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
melody = mg.construct_melody_to_midi(seed2, 1800, SEQUENCE_LENGTH, 0.3)
print(melody)
mg.save_melody(melody,file_name="generated7.mid")



In [40]:
seeds=[
    "59 _ 59 _ 65 _ _ 67 _ _ 67 _ _ _  77 _ 74 _ 83 _ ",
    "74 _ 74 _ 69 _ 59 _ 71 71 _ 72 _ 77 _ _  ",
    "78 _ _ 64 _ _ 83 _ _ _ _ _ ",
    "65 _ _ 65 _ _ 83  _ _ 83 _ 83 _ 77 _ 67 _ _ 74",

    "60 _ 60 _ _ 59 _ 77 _ ",
    "64 _ _ 71 _ _ 65 _ 65",
    "79 _ _ 60 _ 60 _ 72 64",

]
index=1

for seed in seeds:
  seed_melody=mg.construct_melody_to_midi(seed, 0, SEQUENCE_LENGTH, 0.1)
  mg.save_melody(seed_melody,file_name="seed_song"+str(index)+".mid")

  melody = mg.construct_melody_to_midi(seed, 2100, SEQUENCE_LENGTH, 0.8)
  print(melody)
  mg.save_melody(melody,file_name="generated_song"+str(index)+".mid")
  index=index+1




1/1 [==============================] - 0s 75ms/step
['59', '_', '59', '_', '65', '_', '_', '67', '_', '_', '67', '_', '_', '_', '77', '_', '74', '_', '83', '_', '65', '_', '62', '_', '_', '_', '_', '_', '_', '_', '65', '_', '62', '_', '_', '_', '_', '_', '_', '_', '65', '_', '65', '_', '64', '_', '_', '_', '67', '_', '64', '_', '_', '_', '62', '_', '_', '_', '_', '_', '62', '_', '_', '_', '_', '_', '_', '_', '_', '_', '71', '_', '_', '_', '69', '_', '_', '_', '_', '_', 'r', '_', '_', '_', '76', '74', '_', '_', '_', '73', '71', '_', '_', '_', '76', '_', '74', '_', '_', '_', 'r', '_', '_', '_', '66', '_', '_', '_', '78', '_', '_', '_', '_', '_', '73', '_', '71', '_', '83', '_', '_', '_', 'r', '_', '78', '_', '78', '_', '78', '_', '78', '_', '_', '_', '_', '_', '73', '_', '74', '_', '73', '_', '71', '_', '_', '_', 'r', 'r', '_', '_/']
1/1 [==============================] - 0s 42ms/step
['74', '_', '74', '_', '69', '_', '59', '_', '71', '71', '_', '72', '_', '77', '_', '_', '74', '_', '_',